## Load modules

In [ ]:
push!(LOAD_PATH, "./DiscreteValueIteration")

addprocs(int(CPU_CORES / 2))

using DiscreteValueIteration
using SCAs, SCAIterators, SCAConst
using GridInterpolations

mdp = SCA()

## Check size of MDP

In [ ]:
function getBytes(x)
   total = 0;
   fieldNames = typeof(x).names;
   if fieldNames == ()
      return sizeof(x);
   else
     for fieldName in fieldNames
        total += getBytes(getfield(x,fieldName));
     end
     return total;
   end
end

println("mdp of type ", typeof(mdp), " takes up ", getBytes(mdp) / 1000.0, " kB")

## Informal validation of state and action iterators and transition function

In [ ]:
testActions = ActionIterator(Actions)
actionCount = 0
for action in testActions
    actionCount += 1
    if actionCount == 1 || actionCount == NActions
        println("action #", actionCount, ": ", typeof(action), " = ", action)
    end # if
end # for action
println("nActions = ", NActions)
println("actionCount = ", actionCount, "\n")

testStates = StateIterator(Xs, Ys, Bearings, Speeds, Speeds)
stateCount = 0
for state in testStates
    stateCount += 1
    if stateCount == 1 || stateCount == NStates
        println("state #", stateCount, ": ", typeof(state), " = ", state)
    end # if
end # for state
println("nStates = ", NStates)
println("stateCount = ", stateCount)

In [ ]:
nextStateIndices, probs = nextStates(mdp, 1, 15)
println("next state indices:\n", nextStateIndices, "\n")
println("probabilities:\n", probs, "\n")
println("probabilities sum to ", sum(probs))

## Parallel solution

In [ ]:
numProcs = int(CPU_CORES / 2)
solver = ParallelSolver(
    numProcs,
    maxIterations = 1000,
    tolerance = 1e-6,
    gaussSiedel = false,
    includeV = true,
    includeQ = true,
    includeA = true)

In [ ]:
policy = solve(solver, mdp, verbose = true)
println("")

In [ ]:
function sharray2array(sharray::SharedArray{Float64, 2})
    result = zeros(solQ.dims)
    for i = 1:solQ.dims[1]
        for j = 1:solQ.dims[2]
            result[i, j] = sharray[i, j]
        end # for j
    end # for i
    return result
end # function sharray2array

In [ ]:
solQ = sharray2array(policy.Q')
save("../data/par-alpha.jld", "solQ", solQ)

## Check against reference solution by visual inspection

In [ ]:
push!(LOAD_PATH, "../src/")
using Pairwise, Multiagent, JLD

d = get_pomdp()
refQ = load("../data/alpha.jld", "alpha")
solQ = load("../data/par-alpha.jld", "solQ")

### Reference policy plot

In [ ]:
viz_pairwise_policy(d, refQ)

### Parallel solver policy plot

In [ ]:
viz_pairwise_policy(d, solQ)